### Notebook 1: Basic Queries
This notebook offers an introduction to the main function responsible for querying the API and the kinds of data available for analysis. It then leads a reader towards other notebooks based on particular interests and characterization specialties. Two classes have been created, one for querying at the library level and another for querying at the sample level. They are laid out as follows:

##### Library Class
The library class contains four important functions:

Library.search_by_ids(ids_list): This is a static function within the library class. It takes a list of library numbers and returns a list of objects associated with each of the libraries, which can be queried as its own instance of the library class.

Library.search_by_composition(only=[],not_including=[],any_of=[]): This is also a static function within the library class. It takes a list of elements within each of the three lists, then returns a list of objects associated with the libraries that have that specific combination of elements. The "only" list allows a user to specify which elements are required to be in a sample, the "not_including" list allows a user to specify which elements are not allowed in a sample, and the "any_of" list allows a user to specify elements which may be in a sample, but for which it is not necessary for all of them to be so.

Library.properties(self): This function returns all relevant properties data about a library within a pandas DataFrame.

Library.spectra(self,which): This function returns either the optical spectra or the x-ray diffraction spectra for all samples in a library, depending on the value of 'which'. The variable 'which' can be set to 'xrd' to get x-ray diffraction spectra or 'optical' to get the ultraviolet reflectance, ultraviolet transmittance, near-infrared reflectance, and the near-infrared transmittance. This data is returned in a pandas DataFrame.
##### Sample Class
The sample class contains four important functions:

Sample.search_by_ids(ids_list): This is a static function within the library class. It takes a list of sample numbers and returns a list of objects associated with each of the samples, which can be queried as its own instance of the library class.

Sample.properties(self): This function returns all relevent properties data about a sample within a pandas DataFrame.

Sample.spectra(self,which): This function returns either the optical spectra or the x-ray diffraction spectra for a particular sample, depending on the value of 'which'. The variable 'which' can be set to 'xrd' to get x-ray diffraction spectra or 'optical' to get the ultraviolet reflectance, ultraviolet transmittance, near-infrared reflectance, and the near-infrared transmittance. This data is returned in a pandas DataFrame.

In [1]:
import sys
import pandas as pd
sys.path.append('../lib')
#Note: When working in Windows environments, use:
#sys.path.append('..\lib')
from library import Library
from sample import Sample
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

Above one sees that the proper modules have now been imported, including the Library and Sample classes discussed above. A brief example is now shown for each of these class functions.

Below is an example of Library.search_by_ids([ids_list]). The result after querying from the list of samples is a list of objects, which are then called within the "for" loop. Using the Library.properties(), we get back a pandas DataFrame. From this DataFrame, we query information for each of the samples: the computer given id, the PDAC number (that is, the chamber it was made in), the number given to the sample by the researcher, and the elements listed as being a part of the sample.

In [2]:
for lib in Library.search_by_ids([7387,10295,7494,7269]):
    print(lib.properties()[['id','pdac','num','elements']])

     id pdac  num     elements
0  7387    4  399  [Cu, S, Sn]
      id pdac   num         elements
0  10295    1  1161  [In, O, Sn, Zn]
     id pdac   num      elements
0  7494    1  1394  [Ta, Sn, Co]
     id pdac   num         elements
0  7269    1  1211  [Co, O, Ni, Zn]


In [3]:
import json, urllib, bs4
from bs4 import BeautifulSoup
url = 'https://htem-api.nrel.gov/api/sample_library?element=C,S'

with urllib.request.urlopen(url) as response:
    data = json.load(response)
    
data

[{'id': 7096,
  'num': 801,
  'pdac': '4',
  'elements': ['C', 'S'],
  'quality': 3,
  'sample_date': None,
  'person_id': 53,
  'has_xrd': 44,
  'has_xrf': 44,
  'has_opt': 0,
  'has_ele': 0,
  'deposition_compounds': ['C', 'S', None],
  'deposition_power': [60, 80, None],
  'deposition_gases': None,
  'deposition_gas_flow_sccm': None,
  'deposition_sample_time_min': 120,
  'deposition_cycles': None,
  'deposition_substrate_material': None,
  'deposition_base_pressure_mtorr': None,
  'deposition_initial_temp_c': 315,
  'sciround': None},
 {'id': 9074,
  'num': 799,
  'pdac': '4',
  'elements': ['C', 'S'],
  'quality': 3,
  'sample_date': None,
  'person_id': 53,
  'has_xrd': 0,
  'has_xrf': 44,
  'has_opt': 0,
  'has_ele': 0,
  'deposition_compounds': ['C', 'S', None],
  'deposition_power': [60, 80, None],
  'deposition_gases': None,
  'deposition_gas_flow_sccm': None,
  'deposition_sample_time_min': 120,
  'deposition_cycles': None,
  'deposition_substrate_material': None,
  'deposit

Suppose we wish to know some basic information about all the samples that contain a certain series of elements. In the example below, we can use the Library.search_by_composition function to look at information for all samples that have titanium, zinc, oxygen, and tin in them (and as an example, we want to ensure that there is no hydrogen present in them). We find four samples, which we can further explore if we so choose.

In [4]:
temp = """ 
import json, urllib
only=['C','S']
not_including=[]
any_of=[]
elt_url = 'https://htem-api.nrel.gov/api/sample_library?element='

for i in only:
    if i == only[-1]:
        elt_url = elt_url+str(i)
    else:
        elt_url = elt_url+str(i)+','
    
print(elt_url)

response = urllib.request.urlopen(elt_url)
print(response)
data = response.read()
print(data)
ids_list = []
#print(json.loads(response.read()))
for i in data:
    elts = str(i['elements'])
    violated = False
    for k in not_including:
        if k in elts:
            violated = True
    l = 0
    for k in only:
        if k in elts:
            l = l+1
    print('L is '+l)
    print('Only is '+len(only))
    if l == len(only) and violated == False:
        ids_list.append(i['id'])
        print(violated)
    else:
        pass
obj_list = []
print(ids_list)
for i in ids_list:
    obj_list.append(Library(i)) """


In [5]:
#Search terms are 
# only = Only the libraries that have these compounds
# not_including = Excludes These Compounds
# any_of = All 


compound = Library.search_by_composition(only= ['C','S'])
#print(len(compound))

for lib in Library.search_by_composition(only= ['C','S']):#only = ['Ti','Zn','O','Sn'], not_including = ['H']):
    print(lib.properties()[['id','elements','pdac','num']])

     id elements pdac  num
0  7096   [C, S]    4  801
     id elements pdac  num
0  9074   [C, S]    4  799
     id elements pdac  num
0  9075   [C, S]    4  800
     id elements pdac  num
0  7103   [C, S]    4  803
     id elements pdac  num
0  6831   [C, S]    4  804
     id elements pdac  num
0  7324   [C, S]    4  805


Suppose we want to know everything there is to know about a certain library, including information like the deposition time, the deposition power, etc. We can see all of this within a single pandas DataFrame using the Library.properties() function. To narrow this down, one may look at just certain columns of the pandas DataFrame (as shown above).

In [6]:
Library(7387).properties()

,id,num,pdac,quality,person_id,sample_date,owner_name,owner_email,xrf_type,sputter_operator,...,deposition_initial_temp_c,box_number,deposition_gases,deposition_substrate_material,deposition_gas_flow_sccm,has_xrd,has_xrf,has_ele,has_opt,data_access
0,7387,399,4,3,52,None,Lauryn Baranowski,l.l.baranowski@gmail.com,smx,None,...,335,7,None,None,None,44,0,0,0,public


Now we can also query the spectra for different libraries, however this usually results in quite a bit of data. The function Library.spectra(self,which) will return the full x-ray diffraction spectrum (which = 'xrd') for each sample or the full optical spectrum (which = 'opt') for each sample. Take note, however, that these commands access a substantial amount of data and are therefore prone to running a bit slower.

In [7]:
Library(7541).spectra(which='xrd')

/Users/ReidMacBookPro/Documents/GitHub/htem-api-examples/notebooks/../lib/library.py:108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['xrd_background_'+str(leveled_position)] = data['xrd_background']
/Users/ReidMacBookPro/Documents/GitHub/htem-api-examples/notebooks/../lib/library.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['xrd_intensity_'+str(leveled_position)] = data['xrd_intensity']
/Users/ReidMacBookPro/Documents/GitHub/htem-api-examples/notebooks/../lib/library.py:107: PerformanceWarning: DataFrame is hig

,xrd_angle_1,xrd_background_1,xrd_intensity_1,xrd_angle_2,xrd_background_2,xrd_intensity_2,xrd_angle_3,xrd_background_3,xrd_intensity_3,xrd_angle_4,...,xrd_intensity_41,xrd_angle_42,xrd_background_42,xrd_intensity_42,xrd_angle_43,xrd_background_43,xrd_intensity_43,xrd_angle_44,xrd_background_44,xrd_intensity_44
0,19.00,16400,16400,19.00,16020,16020,19.00,11590,11590,19.00,...,6853,19.00,7346,7346,19.00,7337,7337,19.00,7819,7819
1,19.05,16090,15420,19.05,16510,16960,19.05,12020,12270,19.05,...,7337,19.05,7960,8121,19.05,7579,7606,19.05,8324,8571
2,19.10,16180,16450,19.10,16610,15050,19.10,12560,12190,19.10,...,7238,19.10,8391,8413,19.10,7621,7357,19.10,8074,9066
3,19.15,16320,16400,19.15,16810,17570,19.15,12970,13700,19.15,...,8429,19.15,8776,9260,19.15,7695,8094,19.15,8043,7719
4,19.20,16670,16150,19.20,17210,17210,19.20,13420,13450,19.20,...,8330,19.20,9249,8832,19.20,7967,7372,19.20,8298,7277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,51.80,15840,17140,51.80,16870,16800,51.80,14670,13130,51.80,...,9384,51.80,11060,9822,51.80,9704,10400,51.80,10860,11030
657,51.85,15490,16370,51.85,16320,15130,51.85,14300,14350,51.85,...,10220,51.85,10790,10740,51.85,9519,8883,51.85,10590,9925
658,51.90,15000,14820,51.90,16030,16830,51.90,14010,14460,51.90,...,9366,51.90,10640,10770,51.90,9225,8993,51.90,10250,10630
659,51.95,14000,13580,51.95,15720,15920,51.95,13880,12980,51.95,...,10590,51.95,10710,9721,51.95,9186,9316,51.95,9969,9143


In [8]:

import urllib, json
import pandas as pd
from urllib.error import HTTPError

url = 'https://htem-api.nrel.gov/api/sample_library/'+str(7541)
#which = 'xrf'
#which = 'xrd'
which = 'opt'
#which = 'ele'



with urllib.request.urlopen(url) as response:
    data = json.load(response)
#response = urllib.request.urlopen(url)
#data = json.loads(response.read())
positions = data['sample_ids']
df = pd.DataFrame()
#display(positions)

for k in positions:
    #print(k)
    try:
        url = 'https://htem-api.nrel.gov/api/sample/'+str(k)
        with urllib.request.urlopen(url) as response:
            data = json.load(response)
        leveled_position = data['position']
        #display(url)
        if which == 'xrf':
            try:
                #b = len(df)
                #for i in range(len(data['xrf_compounds'])):
                #    df.at[b,'xrf_concentrations_'+(data['xrf_compounds'][i])] = data['xrf_concentration'][i]     
                df['xrf_compounds_'+str(leveled_position)] = data['xrf_compounds']
                df['xrf_concentrations_'+str(leveled_position)] = data['xrf_concentration']
            except:
                #display('No xrf data')
                pass
        elif which == 'xrd': 
            try:
                df['xrd_angle_'+str(leveled_position)] = data['xrd_angle']
                df['xrd_background_'+str(leveled_position)] = data['xrd_background']
                df['xrd_intensity_'+str(leveled_position)] = data['xrd_intensity']
            except:
                #display('No xrd data')
                pass
        elif which == 'opt':
            info_df = pd.DataFrame()
            extra_df = pd.DataFrame()
            try:
                info_df['peak_count_'+str(leveled_position)] = list([data['peak_count']])
                info_df['opt_average_vis_trans_'+str(leveled_position)] = list([data['opt_average_vis_trans']])
                info_df['opt_direct_bandgap_'+str(leveled_position)] = list([data['opt_direct_bandgap']])
                
                extra_df['xyz_mm_'+str(leveled_position)] = data['xyz_mm']
            except:
                pass
            uvir_df = pd.DataFrame()
            try:
                uvir_df['opt_uvir_wavelength_'+str(leveled_position)] = data['opt_uvir_wavelength']
                uvir_df['opt_uvir_response_'+str(leveled_position)] = data['opt_uvir_response']
            except KeyError: #No uvir available
                pass
            uvit_df = pd.DataFrame()
            try:
                uvit_df['opt_uvit_wave_'+str(leveled_position)] = data['opt_uvit_wavelength']
                uvit_df['opt_uvit_response_'+str(leveled_position)] = data['opt_uvit_response']
            except KeyError: #No uvit available
                pass
            nirr_df = pd.DataFrame()
            try:
                nirr_df['opt_nirr_wavelength_'+str(leveled_position)] = data['opt_nirr_wavelength']
                nirr_df['opt_nirr_response_'+str(leveled_position)] = data['opt_nirr_response']
            except KeyError: #No nirr available
                pass
            nirt_df = pd.DataFrame()
            try:
                nirt_df['opt_nirt_wavelength_'+str(leveled_position)] = data['opt_nirt_wavelength']
                nirt_df['opt_nirt_response_'+str(leveled_position)] = data['opt_nirt_response']
            except KeyError: #No nirt available
                pass
            opt_df = pd.DataFrame()
            try:
                opt_df['opt_energy_'+str(leveled_position)] = data['opt_energy']
                opt_df['opt_wavelength_'+str(leveled_position)] = data['opt_wavelength']
                opt_df['opt_normalized_transmittance_'+str(leveled_position)] = data['opt_normalized_transmittance']
                opt_df['opt_absorption_coefficient_'+str(leveled_position)] = data['opt_absorption_coefficient']
            except: #No opt available
                #display('No optical data')
                pass
            pos_df = pd.concat([info_df,extra_df,uvir_df,uvit_df,nirr_df,nirt_df,opt_df],axis=1) 
            df = pd.concat([df,pos_df],axis=1)
        elif which == 'ele':
            ele_df = pd.DataFrame()
            info_df = pd.DataFrame()
            try:
                ele_df['fpm_voltage_volts_'+str(leveled_position)] = data['fpm_voltage_volts'] #5
                ele_df['fpm_current_amps_'+str(leveled_position)] = data['fpm_current_amps'] #5
                
                info_df['fpm_sheet_resistance_'+str(leveled_position)] = list([data['fpm_sheet_resistance']]) #1 
                info_df['fpm_standard_deviation_'+str(leveled_position)] = list([data['fpm_standard_deviation']]) #1
                info_df['fpm_resistivity_'+str(leveled_position)] = list([data['fpm_resistivity']]) #1
                info_df['fpm_conductivity_'+str(leveled_position)] = list([data['fpm_conductivity']]) #1
                info_df['absolute_temp_c_'+str(leveled_position)] = list([data['absolute_temp_c']]) 
                #if data['STRING'] == int else 
            except:
                #display('No electrical data')
                pass
            #display(ele_df)
            #display(info_df)
            pos_df = pd.concat([ele_df,info_df],axis=1)
            df = pd.concat([df,pos_df],axis=1)
        else:
            df = pd.DataFrame()
    except HTTPError as err: #Data missing. Moving on.
        if err.code == 400:
            #print('Sample not found')
            continue

if df.empty:
    print('Empty')   
else:
    display(df)     #Need to look at what information I need. Probably just grab eveything and put it in the databases

#print(df)
#There is the potential to replace this with mvl_optical or mvl_xrd, 
#but these seem to be broken at the moment...

#response = urllib.urlopen(url)
#data = json.loads(response.read())


,peak_count_1,opt_average_vis_trans_1,opt_direct_bandgap_1,xyz_mm_1,opt_uvir_wavelength_1,opt_uvir_response_1,opt_uvit_wave_1,opt_uvit_response_1,opt_nirr_wavelength_1,opt_nirr_response_1,...,opt_uvit_wave_44,opt_uvit_response_44,opt_nirr_wavelength_44,opt_nirr_response_44,opt_nirt_wavelength_44,opt_nirt_response_44,opt_energy_44,opt_wavelength_44,opt_normalized_transmittance_44,opt_absorption_coefficient_44
0,207944.0,0.457,3.684,16.920,300.0,0.2098,300.0,0.19950,899.9,0.3212,...,300.0,0.01272,899.9,0.4162,898.1,0.4818,1.1270,1100.0,0.778800,0.778800
1,NaN,NaN,NaN,17.470,301.0,0.2017,301.0,0.01726,901.6,0.3187,...,301.0,0.19560,901.6,0.4194,904.9,0.4795,4.1330,300.0,0.003437,0.003437
2,NaN,NaN,NaN,7.994,302.0,0.2017,302.0,0.19140,903.4,0.3212,...,302.0,0.18230,903.4,0.4154,911.7,0.4836,4.1140,301.4,0.056240,0.056240
3,NaN,NaN,NaN,NaN,303.0,0.1985,303.0,0.08356,905.1,0.3266,...,303.0,0.12130,905.1,0.4211,918.4,0.4861,4.0950,302.8,0.043670,0.043670
4,NaN,NaN,NaN,NaN,304.0,0.1949,304.0,0.22180,906.9,0.3158,...,304.0,0.05655,906.9,0.4200,925.2,0.4884,4.0770,304.2,0.020100,0.020100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7550,1642.0,0.775200,0.775200
967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7543,1644.0,0.769500,0.769500
968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7537,1645.0,0.766100,0.766100
969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7531,1647.0,0.763900,0.763900


In [9]:
Library(7541).spectra(which='opt')

,peak_count_1,opt_average_vis_trans_1,opt_direct_bandgap_1,xyz_mm_1,opt_uvir_wavelength_1,opt_uvir_response_1,opt_uvit_wave_1,opt_uvit_response_1,opt_nirr_wavelength_1,opt_nirr_response_1,...,opt_uvit_wave_44,opt_uvit_response_44,opt_nirr_wavelength_44,opt_nirr_response_44,opt_nirt_wavelength_44,opt_nirt_response_44,opt_energy_44,opt_wavelength_44,opt_normalized_transmittance_44,opt_absorption_coefficient_44
0,207944.0,0.457,3.684,16.920,300.0,0.2098,300.0,0.19950,899.9,0.3212,...,300.0,0.01272,899.9,0.4162,898.1,0.4818,1.1270,1100.0,0.778800,0.778800
1,NaN,NaN,NaN,17.470,301.0,0.2017,301.0,0.01726,901.6,0.3187,...,301.0,0.19560,901.6,0.4194,904.9,0.4795,4.1330,300.0,0.003437,0.003437
2,NaN,NaN,NaN,7.994,302.0,0.2017,302.0,0.19140,903.4,0.3212,...,302.0,0.18230,903.4,0.4154,911.7,0.4836,4.1140,301.4,0.056240,0.056240
3,NaN,NaN,NaN,NaN,303.0,0.1985,303.0,0.08356,905.1,0.3266,...,303.0,0.12130,905.1,0.4211,918.4,0.4861,4.0950,302.8,0.043670,0.043670
4,NaN,NaN,NaN,NaN,304.0,0.1949,304.0,0.22180,906.9,0.3158,...,304.0,0.05655,906.9,0.4200,925.2,0.4884,4.0770,304.2,0.020100,0.020100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7550,1642.0,0.775200,0.775200
967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7543,1644.0,0.769500,0.769500
968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7537,1645.0,0.766100,0.766100
969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.7531,1647.0,0.763900,0.763900


In [10]:
Library(7541).spectra(which='xrf')

,xrf_compounds_1,xrf_concentrations_1,xrf_compounds_2,xrf_concentrations_2,xrf_compounds_3,xrf_concentrations_3,xrf_compounds_4,xrf_concentrations_4,xrf_compounds_5,xrf_concentrations_5,...,xrf_compounds_40,xrf_concentrations_40,xrf_compounds_41,xrf_concentrations_41,xrf_compounds_42,xrf_concentrations_42,xrf_compounds_43,xrf_concentrations_43,xrf_compounds_44,xrf_concentrations_44
0,NiO,11.10,NiO,6.518,NiO,6.741,NiO,7.227,NiO,8.675,...,NiO,10.64,NiO,15.21,NiO,11.17,NiO,12.85,NiO,15.27
1,CoO,71.06,CoO,71.960,CoO,68.090,CoO,67.640,CoO,62.720,...,CoO,63.63,CoO,49.52,CoO,52.31,CoO,52.53,CoO,46.02
2,ZnO,17.84,ZnO,21.530,ZnO,25.170,ZnO,25.130,ZnO,28.600,...,ZnO,25.73,ZnO,35.27,ZnO,36.52,ZnO,34.62,ZnO,38.71


In [11]:
Library(7541).spectra(which='ele')

,fpm_voltage_volts_1,fpm_current_amps_1,fpm_sheet_resistance_1,fpm_standard_deviation_1,fpm_resistivity_1,fpm_conductivity_1,absolute_temp_c_1,fpm_voltage_volts_2,fpm_current_amps_2,fpm_sheet_resistance_2,...,fpm_resistivity_43,fpm_conductivity_43,absolute_temp_c_43,fpm_voltage_volts_44,fpm_current_amps_44,fpm_sheet_resistance_44,fpm_standard_deviation_44,fpm_resistivity_44,fpm_conductivity_44,absolute_temp_c_44
0,-0.004145,-5.000000e-08,416900.0,755.1,4.478,0.2233,500.0,-0.004639,-5.000000e-08,459000.0,...,3.984,0.251,500.0,-0.004928,-5.000000e-08,457100.0,626.7,3.679,0.2718,500.0
1,-0.001868,-2.500000e-08,NaN,NaN,NaN,NaN,NaN,-0.002103,-2.500000e-08,NaN,...,NaN,NaN,NaN,-0.002409,-2.500000e-08,NaN,NaN,NaN,NaN,NaN
2,0.000438,0.000000e+00,NaN,NaN,NaN,NaN,NaN,0.000398,0.000000e+00,NaN,...,NaN,NaN,NaN,0.000127,0.000000e+00,NaN,NaN,NaN,NaN,NaN
3,0.002758,2.500000e-08,NaN,NaN,NaN,NaN,NaN,0.002928,2.500000e-08,NaN,...,NaN,NaN,NaN,0.002623,2.500000e-08,NaN,NaN,NaN,NaN,NaN
4,0.005039,5.000000e-08,NaN,NaN,NaN,NaN,NaN,0.005504,5.000000e-08,NaN,...,NaN,NaN,NaN,0.005164,5.000000e-08,NaN,NaN,NaN,NaN,NaN


Many of the same techniques used on an entire 44-sample library may also be used on a single sample. Data may be queried just as before, however the information will be specific to a sample instead of a library. Below is an example of the Sample.search_by_ids(id_list) function, which returns a list of objects for each position.

In [12]:
for lib in Sample.search_by_ids([213789]):  #300999,311733,
    #print(lib.properties())
    print(lib.properties()[['id','xrf_compounds','xrf_concentration','thickness']])

       id xrf_compounds xrf_concentration  thickness
0  213789    [MnO, CrO]          [0, 100]    0.05865


The code segment above also makes use of the Sample.properties(self) function. Just as with the Library class, this returns all information relevant to this particular sample, formatted within a pandas DataFrame.

In [13]:
Sample(213789).properties()

,id,sample_library_id,position,thickness,xrf_elements,xrf_compounds,xrf_concentration,xrd_angle,xrd_background,xrd_intensity,...,opt_uvit_wavelength,opt_uvit_response,opt_nirr_wavelength,opt_nirr_response,opt_nirt_wavelength,opt_nirt_response,opt_energy,opt_wavelength,opt_normalized_transmittance,opt_absorption_coefficient
0,213789,6880,7,0.05865,"[Mn, O, Cr]","[MnO, CrO]","[0, 100]","[19, 19.05, 19.1, 19.15, 19.2, 19.25, 19.3, 19...","[75680, 79030, 80450, 82090, 83750, 85430, 865...","[75680, 79890, 79840, 81100, 85180, 85650, 878...",...,"[300, 301, 302, 303, 304, 305, 306, 307, 308, ...","[0.09741, 0.1955, 0.2168, 0.1505, 0.2273, 0.20...",None,None,None,None,"[4.133, 4.12, 4.106, 4.092, 4.079, 4.066, 4.05...","[300, 301, 302, 303, 304, 305, 306, 307, 308, ...","[0.02565, 0.05338, 0.06606, 0.04817, 0.07137, ...","[0.02565, 0.05338, 0.06606, 0.04817, 0.07137, ..."


In the same way that one queries the spectra for an entire library, one can just as easily query a single sample for either x-ray diffraction or optical spectra. Note that the near-infrared spectra within the optical DataFrames are significantly shorter, so the result is that the DataFrame gets padded with Null values within the column.

In [14]:
Sample(234892).spectra('opt') 

,peak_count_1,opt_average_vis_trans_1,opt_direct_bandgap_1,xyz_mm_1,opt_uvir_wavelength_1,opt_uvir_response_1,opt_uvit_wave_1,opt_uvit_response_1,opt_nirr_wavelength_1,opt_nirr_response_1,opt_nirt_wavelength_1,opt_nirt_response_1,opt_energy_1,opt_wavelength_1,opt_normalized_transmittance_1,opt_absorption_coefficient_1
0,207944.0,0.4401,3.027,16.920,300.0,0.1863,300.0,0.24610,899.9,0.3193,898.1,0.4789,4.1330,300.0,0.06632,0.06632
1,NaN,NaN,NaN,13.470,301.0,0.1759,301.0,0.20060,901.6,0.3196,904.9,0.4768,4.1140,301.4,0.03930,0.03930
2,NaN,NaN,NaN,7.994,302.0,0.1745,302.0,0.03935,903.4,0.3200,911.7,0.4812,4.0950,302.8,0.01646,0.01646
3,NaN,NaN,NaN,NaN,303.0,0.1762,303.0,0.05387,905.1,0.3151,918.4,0.4814,4.0770,304.2,0.04573,0.04573
4,NaN,NaN,NaN,NaN,304.0,0.1738,304.0,0.16220,906.9,0.3187,925.2,0.4831,4.0580,305.6,0.03495,0.03495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7550,1642.0,0.72490,0.72490
967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7543,1644.0,0.72660,0.72660
968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7537,1645.0,0.72150,0.72150
969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7531,1647.0,0.71980,0.71980


In [20]:
Sample(234892).spectra('ele')   

,fpm_voltage_volts_1,fpm_current_amps_1,fpm_sheet_resistance_1,fpm_standard_deviation_1,fpm_resistivity_1,fpm_conductivity_1,absolute_temp_c_1
0,-0.004639,-5.000000e-08,459000.0,1295.0,4.293,0.2329,500.0
1,-0.002103,-2.500000e-08,NaN,NaN,NaN,NaN,NaN
2,0.000398,0.000000e+00,NaN,NaN,NaN,NaN,NaN
3,0.002928,2.500000e-08,NaN,NaN,NaN,NaN,NaN
4,0.005504,5.000000e-08,NaN,NaN,NaN,NaN,NaN


This concludes the explanation of the Python classes used to query data from the API.